# NOAA : Extraction de la pluviométrie

Dataset :
- Dossier /demo/noaa/noaa-raw-data
- Format du fichier : "STATION","DATE","LATITUDE","LONGITUDE","ELEVATION","NAME","PRCP","PRCP_ATTRIBUTES"
- Une ligne par station météo

Exercice :
- Récupérer le code pays sur 2 lettres et la placer dans une nouvelle colonne "COUNTRY"
- Supprimer les lignes où PRCP est null
- Regrouper les relevés des stations par pays
- Calculer la moyenne de pluie ("PRCP"), la somme, le maximum, l'écart type, le nombre de stations météo
- Trier par date et par pays
- Sauver dans '/stagiaire/votre_dossier/noaa/daily_rain_by_country'

In [2]:
#!hdfs dfs -ls /demo/noaa/noaa-raw-data

In [2]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import shutil

In [4]:
DATASET_FOLDER = '/demo/noaa/noaa-raw-data'
SPARK_MASTER = 'spark://localhost:7077'
APP_NAME = 'Group daily rain by Country - bco'

noaa_csv_path = DATASET_FOLDER 
output = '/stagiaire/430-Benoit/noaa/daily_rain_by_country_2'

# Create Spark session
spark = SparkSession.builder.master(SPARK_MASTER).appName(APP_NAME).getOrCreate()
# Convert list to data frame
df = spark.read.format('csv').option('header',True).option('multiLine', True).load(noaa_csv_path)
print("done")

done


In [5]:
sc = spark.sparkContext

In [6]:
df.createOrReplaceTempView('tbl')

In [7]:
df2 = spark.sql("""
    select right(name, 2) as COUNTRY, date, count(prcp) cnt, avg(prcp) avg_prcp, sum(prcp) sum_prcp, max(prcp) max_prcp, stddev(prcp) stddev_prcp
    from tbl
    where PRCP is not null
    group by country, date
    order by date desc, country
""")

In [8]:
df2.show()

+-------+----------+---+------------------+--------+--------+------------------+
|COUNTRY|      date|cnt|          avg_prcp|sum_prcp|max_prcp|       stddev_prcp|
+-------+----------+---+------------------+--------+--------+------------------+
|     BE|2020-10-30|  1|               0.0|     0.0|       0|               NaN|
|     FR|2020-10-30| 71| 4.028169014084507|   286.0|      30| 6.661127813100026|
|     RI|2020-10-30|  3|39.666666666666664|   119.0|      76|31.659648345067467|
|     TZ|2020-10-30|  2|             127.0|   254.0|     254|179.60512242138307|
|     BE|2020-10-29|  1|              28.0|    28.0|      28|               NaN|
|     FR|2020-10-29| 71|19.267605633802816|  1368.0|     122|27.431971829797888|
|     RI|2020-10-29|  3|               0.0|     0.0|       0|               0.0|
|     TZ|2020-10-29|  2|              23.0|    46.0|      41| 25.45584412271571|
|     BE|2020-10-28|  1|              46.0|    46.0|      46|               NaN|
|     FR|2020-10-28| 71|31.9

In [9]:
df2.write.csv(output, header=True)

In [10]:
sc.stop()